In [9]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_predict, KFold

In [10]:
data = pd.read_excel('GFA_Dataset.xlsx')

In [11]:
X = data.drop('label', axis=1)
y_true = data['label']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y_true,test_size=0.10)

In [13]:
from sklearn.model_selection import GridSearchCV

model = xgb.XGBClassifier(colsample_bytree = 1, eta = 0.3, gamma = 0.1, max_depth = 7, min_child_weight = 1, subsample = 1)

# param_grid = {
#     'eta': [0.01, 0.1, 0.3], 
#     'max_depth': [3, 5, 7], 
#     'subsample': [0.6, 0.8, 1.0], 
#     'colsample_bytree': [0.6, 0.8, 1.0], 
#     'gamma': [0, 0.1, 0.2],  
#     'min_child_weight': [1, 2, 3], 
# }

# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10)

# grid_search.fit(X_train, y_train)

# print("最优参数组合：", grid_search.best_params_)
# print("最优得分：", grid_search.best_score_)

In [14]:
kf = KFold(n_splits=10, shuffle=True)
confusion_matrices = []
classification_reports = []

In [15]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_true.iloc[train_index], y_true.iloc[test_index]

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)

    confusion_matrices.append(cm)
    classification_reports.append(cr)

In [ ]:
for i, (cm, cr) in enumerate(zip(confusion_matrices, classification_reports)):
    print(f"Fold {i+1}:")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    print("-" * 50)

In [ ]:
average_cm = np.mean(confusion_matrices, axis=0)
y_pred_all = cross_val_predict(model, X, y_true)
average_cr = classification_report(y_true, y_pred_all)
print("Average Confusion Matrix:")
print(average_cm)
print('Accary:', (cm[1,1]+cm[0,0])/(cm[1,1]+cm[0,1]+cm[0,0]+cm[1,0]))

In [ ]:
tn, fp, fn, tp = average_cm.ravel()

print("True Positive (TP):", tp)
print("True Negative (TN):", tn)
print("False Positive (FP):", fp)
print("False Negative (FN):", fn)
print("True Positive Rate (TPR):", tp/ (tp + fn))
print("False Positive Rate (FPR):", fp/ (tn + fp))
print("True Negative Rate (TNR):",  tn / (tn + fp))
print("False Negative Rate (FNR):", fn / (fn + tp))